In [1]:
%run ./apiMethods.ipynb
%run ./arenaCrawlers.ipynb

In [2]:
def createAllTMsJSONandId(allTms):
    allTmJsonsWithid = []
    for machine in allTms:
        index = ((int(machine)-1)*18) + 1
        allTmJsonsWithid.append((requestMachine(index), machine))
    return allTmJsonsWithid


In [3]:
def createAllMoveJSONfromTMJSON(allTmJson):
    allMovesByTmJsons = []
    for tmJson in allTmJson:
        allMovesByTmJsons.append((requestMove(tmJson[0]['move']['name']), tmJson[1] ) )
    return allMovesByTmJsons

In [4]:
def createIdToMoveNameAndAllowedPokemon(allMoveJsonAndId, encounterablePokemonTable):
    idObject = {}
    for moveJson in allMoveJsonAndId:
        tmId = moveJson[1]
        moveJson = moveJson[0]
        moveName = moveJson['name']
        allowedPoke = []
        for pokemon in moveJson['learned_by_pokemon']:
            name = pokemon['name']
            if(encounterablePokemonTable.name.str.contains(name).any()):
                allowedPoke.append(name)
        idObject[tmId] = {"name":moveName,'allowedPoke':allowedPoke} 
    return idObject
                 #  pokemonToTM[name].append(moveName)

In [20]:
def appendTmMovesToMovesOfInterestCSV(moveJsonsWithId, moveTable):
    movesToAppend = []
    movesOfInterestColumNames = ['name', 'moveType', 'damageClass', 'power', 'pp', 'priority', 'accuracy']
    print(moveTable.name)
    for moveJsonWithId in moveJsonsWithId:
        moveJson = moveJsonWithId[0]
        name = moveJson['name']
        if name not in moveTable.name:
            # extracts moves data to CSV format
            moveObject = produceMoveWithoutFetch(moveJson)
            moveCSVformat = [moveObject['name'],moveObject['moveType'],moveObject['damageClass'],moveObject['power'],moveObject['pp'],moveObject['priority'],moveObject['accuracy']]
            moveTable = moveTable.append({'name':moveObject['name'], 'moveType':moveObject['moveType'],'damageClass': moveObject['damageClass'],'power': moveObject['power'],'pp': moveObject['pp'], 'priority':moveObject['priority'],'accuracy':moveObject['accuracy']},ignore_index=True)
    return moveTable
            # ----- #
    #TODO WRITE table ToFile 
    #index=False

In [30]:
def writeTMtoCSV(game):
    allTmJson = createAllTMsJSONandId(POSSIBLE_TM_ARENA9)
    allMoveJson = createAllMoveJSONfromTMJSON(allTmJson)

    for i in range(0, len(machineArrays) ):
        arrayIndex = i
        arenaIndex = i+1
        print('ARENA ' , arenaIndex)
        pathMoves = f'Data/{game}/arena-{arenaIndex}/MovesOfInterest.csv'
        pathPokemon = f'Data/{game}/arena-{arenaIndex}/EncounterablePokemon.csv'
        movesOfInterestTable = pd.read_csv(pathMoves)
        movesOfInterestTable = appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
        appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
        print(movesOfInterestTable.tail())
        
        encounterablePokemonTable = pd.read_csv(pathPokemon)    
        
        pokeToNewMoves = {}
        tmIdToMoveAndAllowedPoke = createIdToMoveNameAndAllowedPokemon(allMoveJson, encounterablePokemonTable)
        
        for tmId in machineArrays[arrayIndex]:
            tmIdNumber = tmId
            allowedPoke = tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']
            moveName = tmIdToMoveAndAllowedPoke[tmIdNumber]['name']
            for pokeName in tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']:
                if pokeName in pokeToNewMoves.keys():
                    pokeToNewMoves[pokeName].append(moveName)
                else:
                     pokeToNewMoves[pokeName] = [moveName]
        newMoveArrays = []
        for name in encounterablePokemonTable.name:
            if name in pokeToNewMoves.keys():
                newMoveArrays.append(pokeToNewMoves[name])
            else:
                newMoveArrays.append([])
        encounterablePokemonTable['movesByTM'] = newMoveArrays
        
        encounterablePokemonTable.to_csv(pathPokemon,index=False) 
        movesOfInterestTable.to_csv(pathMoves,index=False) 

In [9]:
def createAllVMsJSONandId(machines):
    allVmJsonsWithid = []
    for machine in machines:
        index = ((int(machine)-1)*18) + 1 +1316
        print(index)
        allVmJsonsWithid.append((requestMachine(index), machine))
    return allVmJsonsWithid

In [27]:
def writeVMtoCSV(game):
    allTmJson = createAllVMsJSONandId(POSSIBLE_VM_ARENA9)
    allMoveJson = createAllMoveJSONfromTMJSON(allTmJson)

    for i in range(0, len(machineArrays) ):
        arrayIndex = i
        arenaIndex = i+1
        print('ARENA ' , arenaIndex)
        pathMoves = f'Data/{game}/arena-{arenaIndex}/MovesOfInterest.csv'
        pathPokemon = f'Data/{game}/arena-{arenaIndex}/EncounterablePokemon.csv'
        movesOfInterestTable = pd.read_csv(pathMoves)
        movesOfInterestTable = appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
        appendTmMovesToMovesOfInterestCSV(allMoveJson, movesOfInterestTable)
        
        encounterablePokemonTable = pd.read_csv(pathPokemon)    
        
        pokeToNewMoves = {}
        tmIdToMoveAndAllowedPoke = createIdToMoveNameAndAllowedPokemon(allMoveJson, encounterablePokemonTable)
        
        for tmId in machineArraysVM[arrayIndex]:
            tmIdNumber = tmId
            allowedPoke = tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']
            moveName = tmIdToMoveAndAllowedPoke[tmIdNumber]['name']
            for pokeName in tmIdToMoveAndAllowedPoke[tmIdNumber]['allowedPoke']:
                if pokeName in pokeToNewMoves.keys():
                    pokeToNewMoves[pokeName].append(moveName)
                else:
                     pokeToNewMoves[pokeName] = [moveName]
        newMoveArrays = []
        for name in encounterablePokemonTable.name:
            if name in pokeToNewMoves.keys():
                newMoveArrays.append(pokeToNewMoves[name])
            else:
                newMoveArrays.append([])
        encounterablePokemonTable['movesByVM'] = newMoveArrays
        
        encounterablePokemonTable.to_csv(pathPokemon,index=False) 
        movesOfInterestTable.to_csv(pathMoves,index=False) 

In [32]:
writeTMtoCSV('blue')

ARENA  1
0              gust
1       sand-attack
2      quick-attack
3         whirlwind
4       wing-attack
           ...     
109             cut
110           flash
111             fly
112            surf
113        strength
Name: name, Length: 114, dtype: object
0              gust
1       sand-attack
2      quick-attack
3         whirlwind
4       wing-attack
           ...     
159       metronome
160      fire-blast
161         fissure
162      sky-attack
163       explosion
Name: name, Length: 164, dtype: object
           name moveType damageClass power  pp  priority accuracy
159   metronome   normal      status  None  10         0     None
160  fire-blast     fire     special   110   5         0       85
161     fissure   ground    physical  None   5         0       30
162  sky-attack   flying    physical   140   5         0       90
163   explosion   normal    physical   250   5         0      100
ARENA  2
0              gust
1       sand-attack
2      quick-attack
3       

In [29]:
writeVMtoCSV('blue')

1317
1389
1335
1353
1371
ARENA  1
             name moveType damageClass  power  pp  priority  accuracy
0            gust   flying     special   40.0  35         0     100.0
1     sand-attack   ground      status    NaN  15         0     100.0
2    quick-attack   normal    physical   40.0  30         1     100.0
3       whirlwind   normal      status    NaN  20        -6       NaN
4     wing-attack   flying    physical   60.0  35         0     100.0
..            ...      ...         ...    ...  ..       ...       ...
104     metronome   normal      status    NaN  10         0       NaN
105    fire-blast     fire     special  110.0   5         0      85.0
106       fissure   ground    physical    NaN   5         0      30.0
107    sky-attack   flying    physical  140.0   5         0      90.0
108     explosion   normal    physical  250.0   5         0     100.0

[109 rows x 7 columns]
0              gust
1       sand-attack
2      quick-attack
3         whirlwind
4       wing-attack
  

In [20]:
 for number in range(1,5):
    index = ((int(number)-1)*18) + 1 +1316
    print(index)

1317
1335
1353
1371


In [12]:
test = pd.read_csv(f'Data/red/arena-1/MovesOfInterest.csv')

In [14]:
test.name

0              gust
1       sand-attack
2      quick-attack
3         whirlwind
4       wing-attack
           ...     
104       metronome
105      fire-blast
106         fissure
107      sky-attack
108       explosion
Name: name, Length: 109, dtype: object